# Perturbation Theory

Perturbation theory is a collection of versatile methods used in many branches of science, and it divides the system into a model part $\hat{H}_0$ which is a known approximation to the real system $\hat{H}$ and a perturbation part $\hat{V}$.

\begin{equation}
\hat{H} = \hat{H}_0 + \lambda\hat{V}
\end{equation}

where $\lambda$ is a parameter that is small enough to guarantee convergence.

Møller-Plesset perturbation theory is a particular case of perturbation theory, where we take the Fock operator $\hat{F}$ as the model operator, and the perturbation operator is given by

\begin{equation}
\hat{V} = \hat{H} - \hat{F}
\end{equation}

As the cornerstone of ab initio quantum chemistry, Hartree--Fock calculation results in the wavefunction $\Phi_{HF}$, the Fock operator $\hat{F}$, ground state energy $E_{HF}$ and the orbital energies $\{\epsilon_i\}$. Thus, the zeroth-order time-independent Schrodinger equation is defined as:

\begin{equation}
\hat{H}^{(0)}\Phi_{0}^{(0)} = E_{0}^{(0)}\Phi_{0}^{(0)}
\end{equation}

here the zeroth--order Hamiltonian $\hat{H}^{(0)}$ is taken as the Fock operator，

\begin{equation}
\hat{H}^{(0)} = \hat{F}
\end{equation}

and the zeroth-order wavefunction $\Phi_{0}$ refers to the Hartree-Fock wavefunction，

\begin{equation}
\Phi_{0} = \Phi_{HF}
\end{equation}

furthermore, the corresponding energy $E^{(0)}_0$ is the sum of orbital energies $\{\epsilon_i\}$, which is the eigenvalues of the Fock operator $\hat{F}$.

\begin{equation}
E^{(0)} = \sum_i^{occ} \epsilon_i
\end{equation}

As the solution of the zeroth-order equation is known, one expects that the Møller-Plesset theory performs on top of the previous result. Therefore we can determine the various nth-order corrections.

## Algorithm

### Step 1: Build The Integral

The pre-computed quantities for the SCF algorithm include:

- The nuclear repulsion energy $E_{nuc}$.

- One-electron integrals overlap $\mathbf{S}$, kinetic energy $\mathbf{T}$, and nuclear attraction $\mathbf{V}$.

- Two-electron integral, the electron repulsion $(\mu\nu\vert\lambda\sigma)$.

Please check out the :ref:`Hamiltonian <Hamiltonian>` section for further information to learn more about the pre-computed quantities.

### Step 2: Optimized the molecular orbital coefficients by SCF Calculation

The Hartree-Fock method is an uncorrelated mean--field theory that offers a qualitative description of chemical systems. Although Hartree--Fock theory is only qualitatively correct, it forms the basis for more accurate models and becomes the cornerstone of ab initio quantum chemistry.

Please check out the :doc:`user_hf` section for further information.

### Step 3: Transformation of atomic orbital to molecular orbital

With the optimized LCAO_MO coefficients, we can transform the operators from the atomic orbital basis to the molecular orbital basis.

For the one electron operators:
\begin{equation}
h_{ij} = \sum_{\mu\nu}C_{\mu}^jC_{\nu}^i \langle\phi_{\mu}\vert\hat{h}\vert\phi_{\nu}\rangle
= \sum_{\mu\nu} C_{\mu}^j C_{\nu}^i F_{\mu\nu}
\end{equation}

For the two electron operators:
\begin{equation}
\langle pq \vert rs\rangle = \sum_{\mu} C_{\mu}^p \left[
\sum_{\nu} C_{\nu}^q \left[
\sum_{\lambda} C_{\lambda}^r \left[
\sum_{\sigma} C_{\sigma}^s 
\langle\mu\nu\vert\lambda\sigma\rangle\right]\right]\right]
\end{equation}

### Step 4: Calculate the Correction Energy to n--th Order

Perturbation theory applies a biased bi--partition of the Hamiltonian $\hat{H}$. The whole Hilbert space $\mathcal{H}$ is also split into two parts: model space $\mathcal{P}$ and the orthogonal space $\mathcal{Q}$.
\begin{equation}
\mathcal{H} = \mathcal{P} \oplus \mathcal{Q}
\end{equation}

The model space $\mathcal{P}$ is spanned by the reference function $\{\Phi_{0}\}$, and the complementary space $\mathcal{Q}$ is spanned by the excitation configurations $\{S, D, T, \dots\}$.

Given the Møller-Plesset perturbation operator $\hat{V}$, with a reasonably accurate reference $\Phi_{HF}$, we can calculate the electron correlation energy to arbitrary order in a non--iterative way. Here list the equation of correlation energy from orders one up to three:

- MP1

\begin{equation}
E_{MP1} = \langle \Phi_0 \vert\hat{V}\vert \Phi_0 \rangle
= \frac{1}{2} \sum_{ij} \langle ij\vert\vert ij\rangle
\end{equation}

- MP2

\begin{equation}
E_{MP2} = \langle \Phi_0 \vert\hat{V}\vert D \rangle
\langle D \vert\hat{V}\vert \Phi_0 \rangle
= \sum_{ij} \sum_{ab} 
\frac{\langle ia\vert jb\rangle (2 \langle ia\vert jb\rangle -
\langle ib\vert ja\rangle)}
{\epsilon_i + \epsilon_j - \epsilon_a - \epsilon_b}
\end{equation}

## Examples

### MP2

Depending on the choice of Møller-Plesset perturbation operator $\hat{V}$, the first order correction energy $E_{MP1}$ might be zero or none zero, but $E_{MP0} + E_{MP1}$ is always equal to the Hartree--Fock energy $E_{HF}$. Hence, the first meaningful correction in Møller--Plesset perturbation theory is second-order energy.

In [2]:
from moha import *

log.output('silent')

geo = [[8,   0.000000000000,  -0.143225816552,   0.000000000000],
    ['h',   1.638036840407,   1.136548822547,  -0.000000000000],
    ["H",  -1.638036840407,   1.136548822547,  -0.000000000000]]

mol = Molecule.build(geo,pg=False)
orb = BasisSet.build(mol,'sto-3g.nwchem')

ham = Hamiltonian.build(mol,orb)
sym = Symmetry(n=10,ms2=2,ipg=0)

hf_solver = HFSolver(ham,sym)
hf_energy,hf_wfn = hf_solver.uhf()